<a href="https://colab.research.google.com/github/Joao-Girotto/Desafio-Python-Marvel/blob/joao/Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalações

In [ ]:
!pip install dotenv

## Importações

In [ ]:
import requests
import hashlib
import os
import time
import pandas as pd
import sqlite3
import json
from dotenv import load_dotenv
from google.colab import userdata

## Pegando as chaves

In [ ]:
# load_dotenv()
# PUBLIC_KEY = os.getenv("Public_Key")
# PRIVATE_KEY = os.getenv("Private_Key")

PRIVATE_KEY = userdata.get("Private_Key")
PUBLIC_KEY = userdata.get("Public_Key")


## Endpoints e Parâmetros

In [ ]:
ts = str(time.time())
to_hash = ts + PRIVATE_KEY + PUBLIC_KEY
hash_md5 = hashlib.md5(to_hash.encode()).hexdigest()

end1 = "https://gateway.marvel.com/v1/public/characters"
end2 = "https://gateway.marvel.com/v1/public/comics"
end3 = "https://gateway.marvel.com/v1/public/series"


params = {
    "apikey": PUBLIC_KEY,
    "ts": ts,
    "hash": hash_md5,
    "limit": 100
}

## Requisição

In [ ]:
arrayCharacters = []
offset = 0

while offset <= 1564:
  params['offset'] = offset
  response = requests.get(end1, params=params)
  data = response.json()
  arrayCharacters.extend(data['data']['results']) # Adiciona os resultados à lista array
  offset += 100
  print(offset)
  print(response.status_code)

# Depois do loop, crie o DataFrame a partir da lista array
df = pd.DataFrame(arrayCharacters)



## Exibição

In [ ]:
df = df[['name', 'id', 'description', 'comics']]
df['comics_available'] = df['comics'].apply(lambda x: x.get('available'))
df['comics_returned'] = df['comics'].apply(lambda x: x.get('returned'))
df['comics'] = df['comics'].apply(lambda x: json.dumps(x))
con = sqlite3.connect('Marvel.db')
df.to_sql("characters", con, if_exists="replace", index=False)
con.close()

## Consultando o Banco de Dados

In [ ]:
con2 = sqlite3.connect('Marvel.db')
df2 = pd.read_sql_query("SELECT id, name, description, comics_available, comics_returned FROM characters", con2)

query = """
SELECT id, name, description, comics_available, comics_returned
FROM characters
WHERE comics_available = comics_returned
AND comics_available >0
ORDER BY comics_returned DESC;
"""

query1 = """
SELECT SUBSTR(name, 1, 1) AS letra_inicial, COUNT(*) AS total_por_letra
FROM characters
GROUP BY letra_inicial
ORDER BY letra_inicial;
"""
query2 = """
SELECT id, name, comics_available
FROM characters
ORDER BY comics_available DESC
LIMIT 10;
"""

df3 = pd.read_sql_query(query, con2)
contagem_por_letra = pd.read_sql_query(query1, con2)
df5 = pd.read_sql_query(query2,con2)

con2.close()

## Resultados da primeira consulta, demonstrando todos os dados

In [ ]:
df2

## INSIGHT 1 Characters - Demonstração de todos os personagens que tem todos seus quadrinhos disponíveis na API

In [ ]:
df3

## INSIGHT 2 Characters - Criando função para selecionar entidades que tenham nome aliterativo

In [ ]:
def verifica_primeiras_letras(nome_completo):
  """Verifica se o nome e o sobrenome de um nome completo começam com a mesma letra."""
  partes_nome = nome_completo.split()
  if len(partes_nome) >= 2:
    nome = partes_nome[0]
    sobrenome = partes_nome[-1] # Pega a última parte como sobrenome
    if nome and sobrenome: # Verifica se nome e sobrenome não são vazios
      return nome[0].lower() == sobrenome[0].lower()
  return False # Retorna False se não for possível verificar (nome curto ou vazio)

# Aplica a função à coluna 'name' e cria uma nova coluna booleana
df['nome_sobrenome'] = df2['name'].apply(verifica_primeiras_letras)

# Para ver os personagens que atendem à condição
df_mesma_letra = df[df['nome_sobrenome']]
df_mesma_letra[['name', 'nome_sobrenome']]

## INSIGHT 3 Characters - Número de entidades por letra

In [ ]:
contagem_por_letra

## INSIGHT 3 Characters - Gráfico

In [ ]:
contagem_por_letra.plot.bar(x='letra_inicial', y='total_por_letra', legend=False)

## INSIGHT 4 Characters - Demonstração dos 10 personagens que tem mais quadrinhos

In [ ]:
df5.plot.bar(x='name', y='comics_available')